In [2]:
import torch
from torch import nn
from torchvision.transforms import transforms
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch.optim import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
image_path = []
labels = []
for i in os.listdir(r"D:\Datasets\Kaggle\afhq"):
    for label in os.listdir(rf"D:\Datasets\Kaggle\afhq\{i}"):
        for image in os.listdir(rf"D:\Datasets\Kaggle\afhq\{i}\{label}"):
            image_path.append(rf"D:\Datasets\Kaggle\afhq\{i}\{label}\{image}")
            labels.append(label)


data_df = pd.DataFrame(zip(image_path, labels), columns = ["image_path", "labels"])



In [5]:
train = data_df.sample(frac = 0.7)
test = data_df.drop(train.index)
val = test.sample(frac=0.5)
test = test.drop(val.index)

print(train.shape)
print(val.shape)
print(test.shape)

(11291, 2)
(2420, 2)
(2419, 2)


In [7]:
label_encoder = LabelEncoder()
label_encoder.fit(data_df["labels"])
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float)
])

In [8]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform = None):
        self.dataframe = dataframe
        self.transform = transform
        self.labels = torch.tensor(label_encoder.transform(dataframe['labels'])).to(device)
    
    def __len__(self):
        return self.dataframe.shape[0]
    
    def __getitem__(self, idx):
        image_path = self.dataframe.iloc[idx, 0]
        label = self.labels[idx]

        image = Image.open[image_path].convert('RGB')

        if self.transform:
            image = self.transform(image).to(device)

        return image, label
